# Homework: Data Wrangling and Cleaning

We all know that cryptocurrencies are all the rage today.  Could we train an algorithm to tell the difference between a webpage about cryptocurrency and a webpage about something else?

This initial assignment goes over some of the basic steps in (1) acquiring data from the web, (2) acquiring tabular data, (3) cleaning and linking data, and (4) training a simple machine learning classifer.  Along the way you'll learn a few of the basic tools, and get a very basic understanding of one way to represent documents.


In [0]:
# Standard pip install...  Put all of your to-install packages here.
# Depending on your configuration, you may need to change pip3 to pip
!pip3 install scrapy
!pip3 install lxml
!pip3 install scikit-learn
!pip3 install swifter

     |████████████████████████████████| 245kB 4.8MB/s 
     |████████████████████████████████| 3.1MB 50.3MB/s 
     |████████████████████████████████| 174kB 37.1MB/s 
     |████████████████████████████████| 2.3MB 40.9MB/s 
     |████████████████████████████████| 3.2MB 41.4MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
  Created wheel for protego: filename=Protego-0.1.15-cp36-none-any.whl size=7733 sha256=9fab7ac67262bda4f083bd66fe834ced52ca6c7036d341d0194c8d8201622812
  Stored in directory: /root/.cache/pip/wheels/72/d1/f2/4e0a2e6d0179c201952b1b3e086a736548605386193cd312f6
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11516 sha256=334076a51958d28e8560b2850450517573cce3661a8d6d5c40b8b73159a818bc
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built protego PyDispatcher
     |████████████████████████████████| 61k

In [0]:
# Standard imports; it's cleaner to put them here so they can be used
# throughout the notebook

import pandas as pd
import numpy as np
from lxml import etree
import sqlite3
import swifter
import urllib
import re

import nltk
from nltk import classify
from nltk import NaiveBayesClassifier
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


## Task 1: Acquiring data for training our system

First let's get some information about what's a cryptocurrency.  For that -- there's always [Wikipedia](https://en.wikipedia.org/wiki/List_of_cryptocurrencies)!

But of course it won't give us the data exactly the way we want it, so we'll need to do a bit of information extraction and data wrangling. We will also try to get current price levels from [Yahoo](https://finance.yahoo.com/cryptocurrencies).

### Task 1.1: Fetch the list of pages from Wikipedia and put it into a dataframe

First we'll get the master table of "known" cryptocurrencies. Use the `read_html()` function from `pandas`. 

In [0]:
# TODO:
# (1) Fetch files from Wikipedia:  https://en.wikipedia.org/wiki/List_of_cryptocurrencies
# (2) Parse into a dataframe called cryptocurrency_df

### BEGIN SOLUTION

cryptocurrency_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_cryptocurrencies')[0]

### END SOLUTION

display(cryptocurrency_df)

,Release,Currency,Symbol,Founder(s),Hash algorithm,Programming language of implementation,"Cryptocurrency blockchain (PoS, PoW, or other)",Notes
0,2009,Bitcoin,"BTC,[4] XBT, ₿",Satoshi Nakamoto[nt 1],SHA-256d[5][6],C++[7],PoW[6][8],The first and most widely used decentralized l...
1,2011,Litecoin,"LTC, Ł",Charlie Lee,Scrypt,C++[11],PoW,One of the first cryptocurrencies to use Scryp...
2,2011,Namecoin,NMC,Vincent Durham[12][13],SHA-256d,C++[14],PoW,"Also acts as an alternative, decentralized DNS."
3,2012,Peercoin,PPC,Sunny King(pseudonym)[citation needed],SHA-256d[citation needed],C++[15],PoW & PoS,The first cryptocurrency to use POW and POS fu...
4,2013,Dogecoin,"DOGE, XDG, Ð",Jackson Palmer& Billy Markus[16],Scrypt[17],C++[18],PoW,Based on the Doge internet meme.
5,2013[citation needed],Gridcoin,GRC,Rob Hälford[citation needed],Scrypt,C++[19],Decentralized PoS,Linked to citizen science through the Berkeley...
6,2013,Primecoin,XPM,Sunny King(pseudonym)[citation needed],1CC/2CC/TWN[21],"TypeScript, C++[22]",PoW[21],Uses the finding of prime chains composed of C...
7,2013,Ripple[23][24],XRP,Chris Larsen &Jed McCaleb[25],ECDSA[26],C++[27],"""Consensus""",Designed for peer to peer debt transfer. Not b...
8,2013,Nxt,NXT,BCNext(pseudonym),SHA-256d[28],Java[29],PoS,Specifically designed as a flexible platform t...
9,2014,Auroracoin,AUR,Baldur Odinsson(pseudonym)[30],Scrypt,C++[31],PoW,Created as an alternative currency for Iceland...


Next, do the same for the following two sites. Yahoo gives a maximum of 100 prices at a time, so this is why we have to have two queries.

In [0]:
# TODO: Make two price dataframes from
# price_1_df: https://finance.yahoo.com/cryptocurrencies/?count=100&offset=0
# price_2_df: https://finance.yahoo.com/cryptocurrencies/?count=100&offset=100

### BEGIN SOLUTION

price_1_df = pd.read_html('https://finance.yahoo.com/cryptocurrencies/?count=100&offset=0')[0]
price_2_df = pd.read_html('https://finance.yahoo.com/cryptocurrencies/?count=100&offset=100')[0]

### END SOLUTION

price_df = price_1_df.append(price_2_df)

display(price_df)

,Symbol,Name,Price (Intraday),Change,% Change,Market Cap,Volume in Currency (Since 0:00 UTC),Volume in Currency (24Hr),Total Volume All Currencies (24Hr),Circulating Supply,52 Week Range,1 Day Chart
0,BTC-USD,Bitcoin USD,9328.5400,125.2500,+1.36%,168.123B,26.325B,26.325B,26.325B,18.022M,NaN,NaN
1,ETH-USD,Ethereum USD,184.4000,-0.6700,-0.36%,19.984B,9.429B,9.429B,9.429B,108.369M,NaN,NaN
2,XRP-USD,XRP USD,0.2968,0.0003,+0.10%,12.837B,1.633B,1.633B,1.633B,43.248B,NaN,NaN
3,BCH-USD,BitcoinCash USD,285.5500,-4.8400,-1.67%,5.165B,3.105B,3.105B,3.105B,18.087M,NaN,NaN
4,USDT-USD,Tether USD,1.0056,0.0001,+0.01%,4.131B,31.127B,31.127B,31.127B,4.108B,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13,BTM1-USD,Bitmark USD,0.0311,0.0000,0.00%,346920,0,0,0,11.155M,NaN,NaN
14,FRST-USD,FirstCoin USD,0.0077,-0.0001,-1.43%,240593,0,0,0,31.208M,NaN,NaN
15,ATB-USD,ATBCoin USD,0.0012,0.0000,+1.95%,51022,42007,42007,42007,42.038M,NaN,NaN
16,MCAP-USD,MCAP USD,0.0046,0.0000,0.00%,48541,0,0,0,10.49M,NaN,NaN


In [0]:
# Quick sanity check 1.1 for cryptocurrency_df: does it have the columns from the Wikipedia table?

if not 'Currency' in cryptocurrency_df:
    raise AssertionError('Expected column called "Currency"')
    
if not 'Founder(s)' in cryptocurrency_df:
    raise AssertionError('Expected column called "Founder(s)"')

display(cryptocurrency_df)

,Release,Currency,Symbol,Founder(s),Hash algorithm,Programming language of implementation,"Cryptocurrency blockchain (PoS, PoW, or other)",Notes
0,2009,Bitcoin,"BTC,[4] XBT, ₿",Satoshi Nakamoto[nt 1],SHA-256d[5][6],C++[7],PoW[6][8],The first and most widely used decentralized l...
1,2011,Litecoin,"LTC, Ł",Charlie Lee,Scrypt,C++[11],PoW,One of the first cryptocurrencies to use Scryp...
2,2011,Namecoin,NMC,Vincent Durham[12][13],SHA-256d,C++[14],PoW,"Also acts as an alternative, decentralized DNS."
3,2012,Peercoin,PPC,Sunny King(pseudonym)[citation needed],SHA-256d[citation needed],C++[15],PoW & PoS,The first cryptocurrency to use POW and POS fu...
4,2013,Dogecoin,"DOGE, XDG, Ð",Jackson Palmer& Billy Markus[16],Scrypt[17],C++[18],PoW,Based on the Doge internet meme.
5,2013[citation needed],Gridcoin,GRC,Rob Hälford[citation needed],Scrypt,C++[19],Decentralized PoS,Linked to citizen science through the Berkeley...
6,2013,Primecoin,XPM,Sunny King(pseudonym)[citation needed],1CC/2CC/TWN[21],"TypeScript, C++[22]",PoW[21],Uses the finding of prime chains composed of C...
7,2013,Ripple[23][24],XRP,Chris Larsen &Jed McCaleb[25],ECDSA[26],C++[27],"""Consensus""",Designed for peer to peer debt transfer. Not b...
8,2013,Nxt,NXT,BCNext(pseudonym),SHA-256d[28],Java[29],PoS,Specifically designed as a flexible platform t...
9,2014,Auroracoin,AUR,Baldur Odinsson(pseudonym)[30],Scrypt,C++[31],PoW,Created as an alternative currency for Iceland...


### Task 1.2 First bit of data Cleaning:  Clean up the schema names.

It turns out that SQL databases often don't like parentheses and spaces in the column names.  Change the column names for the appropriate columns, by 

1. removing the parts in parentheses
2. trimming any blank spaces before or after the names
3. inserting underscores for spaces.  

Hint: there are functions called `trim`, `strip`, `find`, `replace`.

In [0]:
# TODO:
# For all column names in cryptocurrency_df, 
# (1) remove anything in parentheses, 
# (2) remove leading and trailing spaces, 
# (3) replace remaining spaces with underscores

### BEGIN SOLUTION

for column in cryptocurrency_df.keys():
    col2 = column.strip().replace(' ', '_')
    if column.find('(') >= 0:
        col2 = column[0:column.find('(')] + column[column.rfind(')') + 1:]
        col2 = col2.strip().replace(' ', '_')
    print (column + ": " + col2)
    cryptocurrency_df.rename(columns={column: col2}, inplace=True)
    
### END SOLUTION

Release: Release
Currency: Currency
Symbol: Symbol
Founder(s): Founder
Hash algorithm: Hash_algorithm
Programming language of implementation: Programming_language_of_implementation
Cryptocurrency blockchain (PoS, PoW, or other): Cryptocurrency_blockchain
Notes: Notes


In [0]:
# Sanity check 1.2 for cryptocurrency_df

for column in cryptocurrency_df.keys():
    if column.find(' ') >= 0:
        raise AssertionError('Forgot to remove a space in "%s"'%column)
    elif column.find('(') >= 0 or column.find(')') >= 0:
        raise AssertionError('Forgot to remove a paren in %s'%column)
        
display(cryptocurrency_df)

,Release,Currency,Symbol,Founder,Hash_algorithm,Programming_language_of_implementation,Cryptocurrency_blockchain,Notes
0,2009,Bitcoin,"BTC,[4] XBT, ₿",Satoshi Nakamoto[nt 1],SHA-256d[5][6],C++[7],PoW[6][8],The first and most widely used decentralized l...
1,2011,Litecoin,"LTC, Ł",Charlie Lee,Scrypt,C++[11],PoW,One of the first cryptocurrencies to use Scryp...
2,2011,Namecoin,NMC,Vincent Durham[12][13],SHA-256d,C++[14],PoW,"Also acts as an alternative, decentralized DNS."
3,2012,Peercoin,PPC,Sunny King(pseudonym)[citation needed],SHA-256d[citation needed],C++[15],PoW & PoS,The first cryptocurrency to use POW and POS fu...
4,2013,Dogecoin,"DOGE, XDG, Ð",Jackson Palmer& Billy Markus[16],Scrypt[17],C++[18],PoW,Based on the Doge internet meme.
5,2013[citation needed],Gridcoin,GRC,Rob Hälford[citation needed],Scrypt,C++[19],Decentralized PoS,Linked to citizen science through the Berkeley...
6,2013,Primecoin,XPM,Sunny King(pseudonym)[citation needed],1CC/2CC/TWN[21],"TypeScript, C++[22]",PoW[21],Uses the finding of prime chains composed of C...
7,2013,Ripple[23][24],XRP,Chris Larsen &Jed McCaleb[25],ECDSA[26],C++[27],"""Consensus""",Designed for peer to peer debt transfer. Not b...
8,2013,Nxt,NXT,BCNext(pseudonym),SHA-256d[28],Java[29],PoS,Specifically designed as a flexible platform t...
9,2014,Auroracoin,AUR,Baldur Odinsson(pseudonym)[30],Scrypt,C++[31],PoW,Created as an alternative currency for Iceland...


### Task 1.3: Joining the tables

We are now going to try to put these two sources of information into one table. The requirement is that we want to make sure that we have an entry for every currency in the Wikipedia list, but not necessarily for every currency in the Yahoo price list. Of the four types of join, two can achieve this requirement. For extra practice, see if you can figure out both correct answers.

#### Task 1.3.1 Attempt #1

In the cell below, join `cryptocurrency_df` and `price_df` using "Name" as the join index of `price_df` and "Currency" as the join index of `cryptocurrency_df`. The result should be named `joined_on_name_df`. Do not make any changes to the data frames yet, even though you may see a problem with joining them now.

In [0]:
# TODO: Join cryptocurrency_df and price_df

### BEGIN SOLUTION
joined_on_name_df = cryptocurrency_df.merge(price_df, left_on="Currency", right_on="Name", how="left")
### END SOLUTION

display(joined_on_name_df)

,Release,Currency,Symbol_x,Founder,Hash_algorithm,Programming_language_of_implementation,Cryptocurrency_blockchain,Notes,Symbol_y,Name,Price (Intraday),Change,% Change,Market Cap,Volume in Currency (Since 0:00 UTC),Volume in Currency (24Hr),Total Volume All Currencies (24Hr),Circulating Supply,52 Week Range,1 Day Chart
0,2009,Bitcoin,"BTC,[4] XBT, ₿",Satoshi Nakamoto[nt 1],SHA-256d[5][6],C++[7],PoW[6][8],The first and most widely used decentralized l...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011,Litecoin,"LTC, Ł",Charlie Lee,Scrypt,C++[11],PoW,One of the first cryptocurrencies to use Scryp...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,Namecoin,NMC,Vincent Durham[12][13],SHA-256d,C++[14],PoW,"Also acts as an alternative, decentralized DNS.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012,Peercoin,PPC,Sunny King(pseudonym)[citation needed],SHA-256d[citation needed],C++[15],PoW & PoS,The first cryptocurrency to use POW and POS fu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,Dogecoin,"DOGE, XDG, Ð",Jackson Palmer& Billy Markus[16],Scrypt[17],C++[18],PoW,Based on the Doge internet meme.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2013[citation needed],Gridcoin,GRC,Rob Hälford[citation needed],Scrypt,C++[19],Decentralized PoS,Linked to citizen science through the Berkeley...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2013,Primecoin,XPM,Sunny King(pseudonym)[citation needed],1CC/2CC/TWN[21],"TypeScript, C++[22]",PoW[21],Uses the finding of prime chains composed of C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2013,Ripple[23][24],XRP,Chris Larsen &Jed McCaleb[25],ECDSA[26],C++[27],"""Consensus""",Designed for peer to peer debt transfer. Not b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2013,Nxt,NXT,BCNext(pseudonym),SHA-256d[28],Java[29],PoS,Specifically designed as a flexible platform t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2014,Auroracoin,AUR,Baldur Odinsson(pseudonym)[30],Scrypt,C++[31],PoW,Created as an alternative currency for Iceland...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# Sanity check 1.3.1 for joined_on_name_df

if len(joined_on_name_df.columns) != 20:
    raise AssertionError('Your joined table has %d columns, an unexpected number.'%len(joined_on_name_df.columns))

#### Task 1.3.2 Cleaning up the names

You may have noticed a mismatch for how the currencies are named between the two data frames. Use the `apply` function to replace the values in the `price_df["Name"]` column so they better match the values in `cryptocurrency_df["Currency"]`.

Then rerun your join from 1.3.1 and name it the same way.

In [0]:
# TODO: Remove Fix Name column in price_df and redo the join

### BEGIN SOLUTION

price_df["Name"] = price_df["Name"].apply(lambda x: x if not isinstance(x,str) or x.find(' ') < 0 else ' '.join(x.strip().split()[:-1]))
joined_on_name_df = cryptocurrency_df.merge(price_df, left_on="Currency", right_on="Name", how="left")

### END SOLUTION

display(joined_on_name_df)

,Release,Currency,Symbol_x,Founder,Hash_algorithm,Programming_language_of_implementation,Cryptocurrency_blockchain,Notes,Symbol_y,Name,Price (Intraday),Change,% Change,Market Cap,Volume in Currency (Since 0:00 UTC),Volume in Currency (24Hr),Total Volume All Currencies (24Hr),Circulating Supply,52 Week Range,1 Day Chart
0,2009,Bitcoin,"BTC,[4] XBT, ₿",Satoshi Nakamoto[nt 1],SHA-256d[5][6],C++[7],PoW[6][8],The first and most widely used decentralized l...,BTC-USD,Bitcoin,9328.5400,125.2500,+1.36%,168.123B,26.325B,26.325B,26.325B,18.022M,NaN,NaN
1,2011,Litecoin,"LTC, Ł",Charlie Lee,Scrypt,C++[11],PoW,One of the first cryptocurrencies to use Scryp...,LTC-USD,Litecoin,59.3400,0.9000,+1.55%,3.773B,3.164B,3.164B,3.164B,63.579M,NaN,NaN
2,2011,Namecoin,NMC,Vincent Durham[12][13],SHA-256d,C++[14],PoW,"Also acts as an alternative, decentralized DNS.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012,Peercoin,PPC,Sunny King(pseudonym)[citation needed],SHA-256d[citation needed],C++[15],PoW & PoS,The first cryptocurrency to use POW and POS fu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,Dogecoin,"DOGE, XDG, Ð",Jackson Palmer& Billy Markus[16],Scrypt[17],C++[18],PoW,Based on the Doge internet meme.,DOGE-USD,Dogecoin,0.0026,0.0000,+0.60%,321.649M,76.098M,76.098M,76.098M,121.857B,NaN,NaN
5,2013[citation needed],Gridcoin,GRC,Rob Hälford[citation needed],Scrypt,C++[19],Decentralized PoS,Linked to citizen science through the Berkeley...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2013,Primecoin,XPM,Sunny King(pseudonym)[citation needed],1CC/2CC/TWN[21],"TypeScript, C++[22]",PoW[21],Uses the finding of prime chains composed of C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2013,Ripple[23][24],XRP,Chris Larsen &Jed McCaleb[25],ECDSA[26],C++[27],"""Consensus""",Designed for peer to peer debt transfer. Not b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2013,Nxt,NXT,BCNext(pseudonym),SHA-256d[28],Java[29],PoS,Specifically designed as a flexible platform t...,NXT-USD,Nxt,0.0131,0.0003,+2.55%,13.066M,542558,542558,542558,999M,NaN,NaN
9,2014,Auroracoin,AUR,Baldur Odinsson(pseudonym)[30],Scrypt,C++[31],PoW,Created as an alternative currency for Iceland...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# Sanity check 1.3.2 for joined_on_name_df

if len(joined_on_name_df[joined_on_name_df["Name"].notna()]) == 0:
    raise AssertionError('Your join did not find any matches. Maybe you did something wrong?')

#### Task 1.3.3: Clean the citations out of the content.

As we saw in lecture, the html processing function converts Wikipedia citations to normal text. You may have noticed that this is keeping at least one of the cryptocurrencies from matching during the join. In the cell below, use `applymap` to remove these citations from the entire `cryptocurrency_df` table. Assume that every instance of "`[`" begins a citation. In this case only, it is okay if you delete everything after the "`[`", including the stuff after "`]`".

Then rerun your join from 1.3.2 and name it the same way. Did you get more matches?

In [0]:
# TODO: Remove citations

### BEGIN SOLUTION

cryptocurrency_df = cryptocurrency_df.applymap(lambda x: \
                                               x if not isinstance(x,str) or x.find('[') < 0 \
                                               else x[0:x.find('[')])

joined_on_name_df = cryptocurrency_df.merge(price_df, left_on="Currency", right_on="Name", how="left")

### END SOLUTION

display(joined_on_name_df)

,Release,Currency,Symbol_x,Founder,Hash_algorithm,Programming_language_of_implementation,Cryptocurrency_blockchain,Notes,Symbol_y,Name,Price (Intraday),Change,% Change,Market Cap,Volume in Currency (Since 0:00 UTC),Volume in Currency (24Hr),Total Volume All Currencies (24Hr),Circulating Supply,52 Week Range,1 Day Chart
0,2009,Bitcoin,"BTC,",Satoshi Nakamoto,SHA-256d,C++,PoW,The first and most widely used decentralized l...,BTC-USD,Bitcoin,9328.5400,125.2500,+1.36%,168.123B,26.325B,26.325B,26.325B,18.022M,NaN,NaN
1,2011,Litecoin,"LTC, Ł",Charlie Lee,Scrypt,C++,PoW,One of the first cryptocurrencies to use Scryp...,LTC-USD,Litecoin,59.3400,0.9000,+1.55%,3.773B,3.164B,3.164B,3.164B,63.579M,NaN,NaN
2,2011,Namecoin,NMC,Vincent Durham,SHA-256d,C++,PoW,"Also acts as an alternative, decentralized DNS.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012,Peercoin,PPC,Sunny King(pseudonym),SHA-256d,C++,PoW & PoS,The first cryptocurrency to use POW and POS fu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,Dogecoin,"DOGE, XDG, Ð",Jackson Palmer& Billy Markus,Scrypt,C++,PoW,Based on the Doge internet meme.,DOGE-USD,Dogecoin,0.0026,0.0000,+0.60%,321.649M,76.098M,76.098M,76.098M,121.857B,NaN,NaN
5,2013,Gridcoin,GRC,Rob Hälford,Scrypt,C++,Decentralized PoS,Linked to citizen science through the Berkeley...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2013,Primecoin,XPM,Sunny King(pseudonym),1CC/2CC/TWN,"TypeScript, C++",PoW,Uses the finding of prime chains composed of C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2013,Ripple,XRP,Chris Larsen &Jed McCaleb,ECDSA,C++,"""Consensus""",Designed for peer to peer debt transfer. Not b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2013,Nxt,NXT,BCNext(pseudonym),SHA-256d,Java,PoS,Specifically designed as a flexible platform t...,NXT-USD,Nxt,0.0131,0.0003,+2.55%,13.066M,542558,542558,542558,999M,NaN,NaN
9,2014,Auroracoin,AUR,Baldur Odinsson(pseudonym),Scrypt,C++,PoW,Created as an alternative currency for Iceland...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# Sanity check 1.3.3 for joined_on_name_df

print("%d matches found"%len(joined_on_name_df[joined_on_name_df["Name"].notna()]))
if len(joined_on_name_df[joined_on_name_df["Name"].notna()]) == 0:
    raise AssertionError('Your join did not find any matches. Maybe you did something wrong?')

13 matches found


#### Task 1.3.4 A Better Column

Look again at `cryptocurrency_df` and `price_df` and select better columns for indexing the join. Consider an `apply` function for the relevant column in `cryptocurrency_df` and for the relevant column in price_df` that you select. 

Name this table `joined_df`. To get the points for this section, you need to match at least as many currencies as our solution.

In [0]:
# TODO: Improve the join by switching to different columns

### BEGIN SOLUTION

cryptocurrency_df["Symbol"] = cryptocurrency_df["Symbol"].apply(lambda x: x if not isinstance(x,str) or x.find(',') < 0 else x.strip().split(',')[0])
price_df["Symbol"] = price_df["Symbol"].apply(lambda x: x if not isinstance(x,str) or x.find('-') < 0 else x.strip().split('-')[0])

joined_df = cryptocurrency_df.merge(price_df, on="Symbol", how="left")

### END SOLUTION

# display(joined_df)

In [0]:
# Sanity check 1.3.4 for joined_df

print("%d matches found"%len(joined_df[joined_df["Name"].notna()]))
if len(joined_df[joined_df["Name"].notna()]) <= len(joined_on_name_df[joined_on_name_df["Name"].notna()]):
    raise AssertionError('Your new join is not better than the old one. Maybe you did something wrong?')

18 matches found


### Task 1.4: Save the cryptocurrency list in a database table

We don't want to continue to hit Wikipedia.org every time we want to consult the list of cryptocurrencies.  Save your `cryptocurrency_df` to sqlite, in a table called `cryptocurrency`.  

**The Dataframe `index` has no particular meaning, so don't save it!**

In [0]:
# TODO: convert cryptocurrency_df to sqlite

conn = sqlite3.connect('local.db')

### BEGIN SOLUTION
cryptocurrency_df.to_sql("cryptocurrency", conn, if_exists="replace", index=False)
### END SOLUTION

In [0]:
# Sanity check 1.4 for sqlite databases

crypto2 = pd.read_sql_query('select * from cryptocurrency', conn)

if 'index' in crypto2:
    raise AssertionError('Please disable the index, since it isn\'t important information')
    
display(crypto2)

,Release,Currency,Symbol,Founder,Hash_algorithm,Programming_language_of_implementation,Cryptocurrency_blockchain,Notes
0,2009,Bitcoin,BTC,Satoshi Nakamoto,SHA-256d,C++,PoW,The first and most widely used decentralized l...
1,2011,Litecoin,LTC,Charlie Lee,Scrypt,C++,PoW,One of the first cryptocurrencies to use Scryp...
2,2011,Namecoin,NMC,Vincent Durham,SHA-256d,C++,PoW,"Also acts as an alternative, decentralized DNS."
3,2012,Peercoin,PPC,Sunny King(pseudonym),SHA-256d,C++,PoW & PoS,The first cryptocurrency to use POW and POS fu...
4,2013,Dogecoin,DOGE,Jackson Palmer& Billy Markus,Scrypt,C++,PoW,Based on the Doge internet meme.
5,2013,Gridcoin,GRC,Rob Hälford,Scrypt,C++,Decentralized PoS,Linked to citizen science through the Berkeley...
6,2013,Primecoin,XPM,Sunny King(pseudonym),1CC/2CC/TWN,"TypeScript, C++",PoW,Uses the finding of prime chains composed of C...
7,2013,Ripple,XRP,Chris Larsen &Jed McCaleb,ECDSA,C++,"""Consensus""",Designed for peer to peer debt transfer. Not b...
8,2013,Nxt,NXT,BCNext(pseudonym),SHA-256d,Java,PoS,Specifically designed as a flexible platform t...
9,2014,Auroracoin,AUR,Baldur Odinsson(pseudonym),Scrypt,C++,PoW,Created as an alternative currency for Iceland...


### Task 1.5: Read the cryptocurrency pages

Now let's take each of the cryptocurrency names and find the associated URL. The names of the currencies were originally clickable links on the [webpage](https://en.wikipedia.org/wiki/List_of_cryptocurrencies) that we made the table from, but unfortunately, `pandas` automatically deleted the URLs. So we have to regenerate them. Feel free to look at that page to see what the correct URL is for each currency.

In the cell below, complete the function `crawl`. The function name, inputs, first line, and last line are provided for you. 

`list_of_urls` should contain the URLs of interest as a list, column of a pandas DataFrame, or some other iterable over strings. 

`prefix` contains a common string that should be added to the beginning every URL in `list_of_urls` before each URL is queried. 

The line `pages = {}` creates an empty dictionary. After running your part of the function `crawl`, `pages` should have currency names as its keys and the corresponding Wikipedia page contents as its values. This is what the function returns.

You have two options for completing this cell:

1. If you want to use `urllib.request.urlopen`, you should then use `read()` and `decode('utf-8')`.

2. If you want to use `scrapy`, follow the process in [this notebook from class](https://www.google.com/url?q=https://drive.google.com/file/d/1VfnlGr_VofdcEqACM2jRu2BwYm0QyTSh/view?usp%3Dsharing&sa=D&ust=1567968915286000&usg=AFQjCNG5iEWgUoA3DrRLhV1TKiT2OXHD1A).

For now, use a `try` statement to catch the errors and print a message that the URL could not be crawled. That is, in this cell we will have a **single rule** and not do any manual cleaning.  If you were doing this at web scale, you would be reluctant to invest a lot of manual effort...

In [0]:
# TODO: Crawl the pages.  
# Trap the errors and figure out what you need to fix (in the cleaning step below)

def crawl(list_of_urls, prefix=""):
    pages = {}
### BEGIN SOLUTION
    for page in list_of_urls:
        try:
            content = urllib.request.urlopen(prefix + page).read()
            pages[page] = content.decode('utf-8')
        except:
            print ('Unable to read ' + page)
### END SOLUTION
    return pages

The following cell passes the currencies in our table to the `crawl` function. You do not need to modify the cell.

In [0]:
# Sanity check 1.5.1 for initial crawl

pages = crawl(cryptocurrency_df['Currency'], 'https://en.wikipedia.org/wiki/')
for page in pages:
    print (page)
    
print ('Total crawl: %d cryptocurrencies'%len(pages))

Unable to read Ether or "Ethereum"
Unable to read Ethereum Classic
Unable to read Bitcoin Cash
Bitcoin
Litecoin
Namecoin
Peercoin
Dogecoin
Gridcoin
Primecoin
Ripple
Nxt
Auroracoin
Dash
NEO
MazaCoin
Monero
NEM
PotCoin
Titcoin
Verge
Stellar
Vertcoin
Tether
Zcash
EOS.IO
Total crawl: 23 cryptocurrencies


Did you get any errors? Did you ever get the wrong URL (and therefore the content from the wrong page)? Fix those two problems in the function `crawl_better` below. This function has the same inputs and outputs as `crawl`, but this time, it is okay if your fixes are specific to these sites. For example, you can try attaching `_(disambiguation)`, pull up that page's `etree.HTML(content)` and look for a link that has the name of the currency plus `' (cryptocurrency)'`.

In [0]:
# TODO: Re-run the crawl, fixing the issues

# Crawl the pages.  You may use urllib.request.urlopen or scrapy
# Assemble the list of results in the list pages.
# Trap the errors and figure out what you need to fix (in the cleaning step below)

def crawl_better(list_of_urls, prefix=""):
    pages = {}
### BEGIN SOLUTION
    for page in list_of_urls:
        try:
            original = page
            page = page.replace('"', '')
            if page.find(' or ') >= 0:
                page = page[page.find(' or ') + 4:]

            print ('Crawling %s'%page)
            content = urllib.request.urlopen(prefix + page).read()

            content = content.decode('utf-8')

            if 'blockchain' not in content:
                url = 'https://en.wikipedia.org/wiki/' + page + '_(disambiguation)'
                print ('Not the right page.  Disambiguating via %s'%url)

                content = urllib.request.urlopen(url).read()

                content_dom = etree.HTML(content)
                xpath = '//a[text()=\"' + page + ' (cryptocurrency)\"]/@href'
                xp = content_dom.xpath(xpath)
                if xp:
                    print ('Correct page is %s'%str(xp[0]))
                    content = urllib.request.urlopen('https://en.wikipedia.org' + str(xp[0])).read()
                else:
                    xpath = '//a[text()=\"' + page + ' (payment protocol)\"]/@href'
                    xp = content_dom.xpath(xpath)
                    if xp:
                        print ('Correct page is %s'%str(xp[0]))
                        content = urllib.request.urlopen('https://en.wikipedia.org' + str(xp[0])).read()
                    else:
                        xpath = '//a[text()=\"' + page + ' (payment network)\"]/@href'
                        xp = content_dom.xpath(xpath)
                        if xp:
                            print ('Correct page is %s'%str(xp[0]))
                            content = urllib.request.urlopen('https://en.wikipedia.org' + str(xp[0])).read()
                        else:
                            raise AssertionError('Could not find disambiguation')
                            
                content = content.decode('utf-8')

            pages[page] = content
        except:
            print ('Unable to read cryptocurrency for %s'%page)

### END SOLUTION
    return pages

As before, the cell below just runs your function and does not need to be modified.

In [0]:
# Sanity check 1.5.2 for better crawl

pages = crawl_better(cryptocurrency_df['Currency'], 'https://en.wikipedia.org/wiki/')

Crawling Bitcoin
Crawling Litecoin
Crawling Namecoin
Crawling Peercoin
Crawling Dogecoin
Crawling Gridcoin
Crawling Primecoin
Crawling Ripple
Not the right page.  Disambiguating via https://en.wikipedia.org/wiki/Ripple_(disambiguation)
Correct page is /wiki/Ripple_(payment_protocol)
Crawling Nxt
Crawling Auroracoin
Crawling Dash
Not the right page.  Disambiguating via https://en.wikipedia.org/wiki/Dash_(disambiguation)
Correct page is /wiki/Dash_(cryptocurrency)
Crawling NEO
Crawling MazaCoin
Crawling Monero
Not the right page.  Disambiguating via https://en.wikipedia.org/wiki/Monero_(disambiguation)
Correct page is /wiki/Monero_(cryptocurrency)
Crawling NEM
Not the right page.  Disambiguating via https://en.wikipedia.org/wiki/NEM_(disambiguation)
Correct page is /wiki/NEM_(cryptocurrency)
Crawling PotCoin
Crawling Titcoin
Crawling Verge
Not the right page.  Disambiguating via https://en.wikipedia.org/wiki/Verge_(disambiguation)
Correct page is /wiki/Verge_(cryptocurrency)
Crawling Ste

### Task 1.6: Sanity-check and fix

Note that sometimes terms in Wikipedia are **ambiguous**, so just following the page doesn't always get what you want.  The Wikipedia page for [Tether](https://en.wikipedia.org/wiki/Tether) does not describe a cryptocurrency.

We can add a data-cleaning rule to check this: every cryptocurrency should mention the term "blockchain".  Here's a sanity check you can use.  If there are any disambiguation pages, you need to go back to Task 1.5 and update your process to crawl the right page.

You do not need to modify this cell.

In [0]:
count_wrong = 0

for page,content in pages.items():
    if isinstance(content, bytes):
        raise AssertionError('Please run decode(\'utf-8\') on the content to decode to a string')
        content = content.decode('utf-8')
        
    if 'blockchain' not in content:
        print(page + ': ' + ' -- did not find blockchain!')
        count_wrong = count_wrong + 1

        
print ('Total crawl: %d cryptocurrencies'%len(pages))

if count_wrong > 0:
    raise AssertionError('Need to follow Wikipedia disambiguation pages on %d items!'%count_wrong)

Total crawl: 24 cryptocurrencies


### Task 1.7: Clean the articles

So far, we have captured HTML content for each Wikipedia article, but HTML is not very easy to read and process. So the next step is to clean up the text in each article. To do that, you need to complete the function definition below. The function name, and input are provided for you. 

The first step is to get a list of paragraphs of content. See our [slides](https://www.google.com/url?q=https://drive.google.com/a/seas.upenn.edu/file/d/163sCi0h5RJAXynE1Vo37bAQtOvcwW_wv/view?usp%3Dsharing&sa=D&ust=1567968915286000&usg=AFQjCNGDBY3SNFEJIh3m5k7GyYmhK2Q52w) on xpath for hints. Then, for each word (string between whitespace characters):

1. Remove the leading and trailing whitespace using `strip()`
2. Remove the word entirely if it is only white space.
3. Remove the word entirely if it is only numerics (you may use `isnumeric()` to test for this).

Finally join the words together into one string with spaces in between using `' '.join()`. The function should return that string (output).

In [0]:
# TODO: Complete the clean_article function, as described above.

def clean_article(content):
### BEGIN SOLUTION
    content_paras = etree.HTML(content).xpath('//p')
    content2 = []
    for content_para in content_paras:
        content2 = content2 + [x.strip() for x in content_para.xpath('.//text()') \
                               if len(x.strip()) > 0 and 
                               not x.strip().isnumeric()]
    return ' '.join(content2)
### END SOLUTION

The following cell assembles our cleaned articles into a DataFrame. You do not need to modify the cell.

In [0]:
pages2 = []
for currency_name, content in pages.items():
    article = clean_article(content)
    pages2.append({'currency': currency_name, 'text': article})

pages_df = pd.DataFrame(pages2)

display(pages_df)

,currency,text
0,Bitcoin,Bitcoin [a] ( ₿ ) is a cryptocurrency . It is ...
1,Litecoin,Litecoin ( LTC or Ł ) is a peer-to-peer crypto...
2,Namecoin,Namecoin ( Symbol : ℕ or NMC ) is a cryptocurr...
3,Peercoin,"Peercoin , also known as PPCoin or PPC , is a ..."
4,Dogecoin,"Dogecoin ( / ˈ d oʊ dʒ k ɔɪ n / DOHJ -koyn , [..."
5,Gridcoin,"Gridcoin implements a ""Proof-of-Research"" (POR..."
6,Primecoin,Primecoin ( sign : Ψ ; code: XPM ) is a crypto...
7,Ripple,"Ripple is a real-time gross settlement system,..."
8,Nxt,Nxt is an open source cryptocurrency and payme...
9,Auroracoin,"Auroracoin (code: AUR, symbol: ᚠ ) is a peer-t..."


# Task 2: Build and run the classifier

Now that we have the cryptocurrency articles processed, it is time to return to the original task of building a classifier that can identify cryptocurrency articles.

## Task 2.1: Get the negative examples.

If we want to build a (supervised) machine learning algorithm to detect content, we need both *positive* and *negative* examples.  In fact we want each successive training example to have an equal probability of being positive or negative.

The following cell runs your `crawl` function from Task 1.5 and your `clean_article` function from Task 1.7. Note: We are using `crawl` not `crawl_better` because you may have included data-specific choices in `crawl_better` that are no longer true.

You do not need to modify this cell.

In [0]:
training = [
    'https://en.wikipedia.org/wiki/Tim_Cook',
    'https://en.wikipedia.org/wiki/The_Great_British_Bake_Off',
    'https://en.wikipedia.org/wiki/Google',
    'https://en.wikipedia.org/wiki/Chan_Zuckerberg_Initiative',
    'https://en.wikipedia.org/wiki/Politics',
    'https://en.wikipedia.org/wiki/Fake_news',
    'https://www.snopes.com/fact-check/social-media-hacker-warning/',
    'https://www.cnn.com/2019/08/31/us/dorian-animals-foster-release-wxc/index.html',
    'https://www.foxnews.com/us/indiana-dispatcher-helps-boy-who-called-911-with-fractions-homework',
    'https://www.usatoday.com/story/tech/talkingtech/2019/08/31/hello-iphone-11-new-features-we-want-apple-next-models/2153565001/',
    'http://theconversation.com/bury-fc-the-economics-of-an-english-football-clubs-collapse-122727',
    'https://fivethirtyeight.com/features/economists-are-bad-at-predicting-recessions/'
]

negative = crawl(training)
negative2 = []
for site, content in negative.items():
    article = clean_article(content)
    negative2.append({'site': site, 'text': article})

negative_df = pd.DataFrame(negative2)
display(negative_df)

,site,text
0,https://en.wikipedia.org/wiki/Tim_Cook,"Timothy Donald Cook (born November 1, 1960) [3..."
1,https://en.wikipedia.org/wiki/The_Great_Britis...,The Great British Bake Off (often abbreviated ...
2,https://en.wikipedia.org/wiki/Google,Google LLC is an American multinational techno...
3,https://en.wikipedia.org/wiki/Chan_Zuckerberg_...,The Chan Zuckerberg Initiative ( CZI ) is a li...
4,https://en.wikipedia.org/wiki/Politics,Politics is a set of activities associated wit...
5,https://en.wikipedia.org/wiki/Fake_news,"Fake news (also known as junk news , pseudo-ne..."
6,https://www.snopes.com/fact-check/social-media...,Snopes needs your help! Learn more . Accepting...
7,https://www.cnn.com/2019/08/31/us/dorian-anima...,"By Madeline Holcombe , CNN Updated 3:20 AM ET,..."
8,https://www.foxnews.com/us/indiana-dispatcher-...,"This material may not be published, broadcast,..."
9,https://www.usatoday.com/story/tech/talkingtec...,Settings Cancel Set Have an existing account? ...


## Task 2.2: Process Document Text

Right now, each Wikipedia article is a single string. This means, we only have one "feature" for the classifier. This is not enough. Tokenization (splitting up the article into words) would transform the data so that we have one feature per word. This probably would give us enough features to train a classifier.

Complete the `get_words` function in the cell below. This function should take a string as input (the raw article).

1. Create an empty list to store the good words.

1. Break the article into sentences using the NLTK sentence tokenizer.

1. Tokenize and part-of-speech tag each sentence.

1. Run the provided `clean_word` function and Porter stemmer on each word.

1. Finally, append the word stem to the list of good words if all of the following are true:
    1. The word stem is of nonzero length.
    2. The word stem has a length less than 20.
    3. The word stem is not a stopword.
    4. The word is a noun.
    5. The word stem is in `vocabulary`. Only apply this rule if `vocabulary` has nonzero length. It has zero length by default.

6. Return the list of good words.

To match our solution, it is important that you do these steps in the given order.

In [0]:
# TODO: Complete the get_words function

sw = set(stopwords.words("english"))
sw.add("'s")
stemmer = PorterStemmer()

def clean_word(word):
    word = word.lower()
    word2 = ''
    for w in word:
        if w.isalpha() or (len(word2) > 0 and w.isnumeric()):
            word2 = word2 + w
    return word2

def get_words(article, vocabulary=[]):
### BEGIN SOLUTION
    article_clean = []
    sentences = nltk.sent_tokenize(article)
    for sentence in sentences:
        tweet_tokens = nltk.word_tokenize(sentence)
        tagged = nltk.pos_tag(tweet_tokens) 
        for (word, tag) in tagged:
            word_lower = clean_word(word)
        
            word_lower = stemmer.stem(word_lower)
            if len(word_lower) == 0: continue
            if len(word_lower) > 20: continue
            if word_lower in sw: continue
            if tag[0] != "N" : continue
            #if tag[0] != "V" : continue
            if len(vocabulary) > 0:
                if word_lower not in vocabulary: continue
            article_clean.append(word_lower)    
    return article_clean
### END SOLUTION  

In [0]:
# Sanity check 2.2 for getting the word stems from articles

print(get_words("to be or not to be"))
print(get_words("He wants to test the functionality of xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx in article 091019."))

[]
['function', 'articl']


## Task 2.3 Train the classifier

Adapt the code from the NLTK lecture notebook to complete the `build_classifier` function. This function takes as input the two column dataframes `positive_df` and `negative_df`, and also an optional vocabulary list. It should run `get_words` on each article in each dataframe, get a frequency distribution from NLTK for each article, assemble the training set for a Naive Bayes classifier in the correct format, train the classifier, and return the trained classifier.

In [0]:
# TODO: Complete the build_classifier function

def build_classifier(positive_df, negative_df, vocabulary=[]):
### BEGIN SOLUTION
    pos_tweets_set = []
    for tweet in pages_df['text']:
        pos_tweets_set.append((nltk.FreqDist(get_words(tweet, vocabulary)), 'positive'))
    neg_tweets_set = []
    for tweet in negative_df['text']:
        neg_tweets_set.append((nltk.FreqDist(get_words(tweet, vocabulary)), 'negative'))
        
    return NaiveBayesClassifier.train(pos_tweets_set + neg_tweets_set)
### END SOLUTION

In [0]:
# Sanity check 2.3 for training the classifier

classifier = build_classifier(pages_df, negative_df)
print(type(classifier))

# This should print <class 'nltk.classify.naivebayes.NaiveBayesClassifier'>

<class 'nltk.classify.naivebayes.NaiveBayesClassifier'>


## Task 2.4: Run the classifier

Below are some sample pages.  Let's see if you can run the model on them.

### Task 2.4.1 Load the test set

Adapt the code from Task 2.1 for the new dataset. Call the final dataframe `inference_df`.

In [0]:
# TODO: Create inference_df

test = [
    'https://fried.com/history-of-bitcoin/',
    'https://news.wharton.upenn.edu/press-releases/2018/06/penn-launches-strategic-collaboration-ripple-accelerate-innovation-blockchain-cryptocurrency/',
    'https://en.wikipedia.org/wiki/Euro',
    'https://ew.com/movies/star-wars-rise-of-skywalker-footage-d23-expo/',
    'https://en.wikipedia.org/wiki/Donald_Trump'
]

### BEGIN SOLUTION
inference = crawl(test)
inference2 = []
for site, content in inference.items():
    article = clean_article(content)
    inference2.append({'site': site, 'text': article})

inference_df = pd.DataFrame(inference2)

### END SOLUTION

In [0]:
# Sanity check 2.4.1 loading the test set

display(inference_df)

,site,text
0,https://fried.com/history-of-bitcoin/,"Follow us! Last updated: August 5th, 2019 Bitc..."
1,https://news.wharton.upenn.edu/press-releases/...,"PHILADELPHIA, PA, June 4, 2018 — The Wharton S..."
2,https://en.wikipedia.org/wiki/Euro,The euro ( sign : € ; code : EUR ) is the offi...
3,https://ew.com/movies/star-wars-rise-of-skywal...,With the new Star Wars: The Rise of Skywalker ...
4,https://en.wikipedia.org/wiki/Donald_Trump,"Donald John Trump (born June 14, 1946) is the ..."


### Task 2.4.2: Inference

Now let's run your classifier over your individual documents. Adapt the code from the NLTK lecture notebook. The function classify should take as input a two column dataframe as we have made previously, the trained classifier, and an optional vocabulary list. It should return a list of booleans. For example, a perfect classifier should return

`classify(inference_df, classifier) = [True, True, False, False, False]`.

Note that you will need to run `get_words` (passing the vocabulary) and then generate an NLTK frequency distribution for each test article.

In [0]:
# TODO: Complete the classify function

def classify(df, classifier, vocabulary=[]):
### BEGIN SOLUTION
    results = []
    for page in df['text']:
        custom_tweet_set = nltk.FreqDist(get_words(page, vocabulary))        
        prob_result = classifier.prob_classify(custom_tweet_set)
        print(prob_result.prob('positve'))
        print(prob_result.prob('negative'))
        if prob_result.max() == "positive" : results.append(True)
        else: results.append(False)
            
    return results
### END SOLUTION

results = classify(inference_df, classifier)
display(results)

0
1.0
0
1.0
0
1.0
0
1.0
0
1.0


[False, False, False, False, False]

In [0]:
# Sanity check 2.4.2 classifier results

if len(results) != 5:
    raise AssertionError('We do not have a classification for each item.')

## Task 2.5: Make the vocabulary and re-classify

So far, our classifier is not very good. This is because it is trying to consider too many words, many of which did or did not occur in the training articles purely by chance. If we restrict the "attention" of the classifier to the most frequent words, it is much more likely to pick up real patterns rather than memorize accidents. We do this by making a vocabulary.

Complete the `make_vocabulary` function below. This function should take as input the two column dataframes `positive_df` and `negative_df`, and also an integer `num`. For the positive dataframe, run `get_words` on each article (without vocabulary), concatenate all of these lists of words together, create an NLTK frequency distribution, and then finally store a list of the `num` most frequent words. Do the same for the negative dataframe. The function should return the `num` most frequent positive words and the `num` most frequent negative words concatenated into one list (2 times `num` words in all).



In [0]:
# TODO: Complete the make_vocabulary function

def make_vocabulary(positive_df, negative_df, num):
### BEGIN SOLUTION
    allpos = []
    for article in positive_df['text']:
        allpos += get_words(article)

    pos_words = list(dict(nltk.FreqDist(allpos).most_common(num)).keys())

    allneg = []
    for article in negative_df['text']:
        allneg += get_words(article)

    neg_words = list(dict(nltk.FreqDist(allneg).most_common(num)).keys())

    return pos_words + neg_words
### END SOLUTION

In [0]:
# Sanity check 2.5.1 see final vocabulary size

vocabulary = make_vocabulary(pages_df, negative_df, 30)
print(len(vocabulary))

60


In [0]:
# Sanity check 2.5 improved classifier results

classifier_with_vocab = build_classifier(pages_df, negative_df, vocabulary)
results = classify(inference_df, classifier_with_vocab, vocabulary)
display(results)